In [0]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

from requests import get
import re

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [2]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file = get('https://raw.githubusercontent.com/ArthurSiqueiraS/RNN-LSTM/master/lfv_texts_plain.txt?token=AHJJ64SA2ARVW2EF47KFBIK5AXELI').text

symbols = ['\n',' ',',','.','!','?','*','(',')','"',':','…','―','@','$','%', "”", "“"]
for symbol in symbols:
  file = file.replace(symbol, "#" + symbol + "#")

file = file.split("#")
file = list(filter(lambda w: w != '' or w == '\n', file))

file_len = len(file)
print('file_len =', file_len)

all_words = list(set(file))
n_words = len(all_words)
print(all_words, n_words)

file_len = 494265
['expandindo', 'Cherries', 'bem-postas', 'birra', 'difíceis', '“', 'barbuda', 'esforçaram', 'descrevê-la', 'ganso', 'gritar', 'atirado', 'sairmos', 'Desisto', 'Montei', 'vomitara', 'contam', 'sonhava', 'fiquei', 'abandonada', 'imprópria', 'concentrar', 'recusa', 'Concluí', 'Manhattan', 'revelarem', 'casou', 'Valdemira', 'construíram', 'cheio', 'deveríamos', 'agricultura', 'procriar', 'FNAC', 'entregando-se', 'burra', 'cientista', 'catedral', 'tamancos', 'lua-', 'Fariam', 'Quais', 'eterno', 'conter', 'subversão', 'Conselhos', 'acabado', 'merecia', 'Caçadores', 'cômoda', 'Brulê', 'bebem', 'acusada', 'selecionando', 'realizaria', 'surfistas', 'agradáveis', 'sente-se', 'ignorante', 'Aceite', 'tranqüilidade', 'salão', 'maquinação', 'Marseillaise', 'imperfeição', 'analfabeto', 'tinha-se', 'feriu', 'sereno', 'trás', 'destaparem', 'chegam', 'cardeais', 'Polidori', 'Desfilamos', 'redobrados', 'abaixarem', 'cidadezinha', 'nem', 'Bloomsbury', 'papelão', 'preven

In [3]:
chunk_len = 30

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

['era', ' ', 'inconcebível', '.', ' ', 'Para', ' ', 'muitos', ',', ' ', 'continua', ' ', 'inconcebível', '.', '\n', '*', '*', '*', '\n', 'A', ' ', 'Tempestade', ' ', 'não', ' ', 'é', ' ', 'exatamente', ' ', 'uma', ' ']


In [4]:
# Converte palavras para uma lista de inteiros

def word_tensor(words):
    tensor = torch.zeros(len(words)).long()
    for i in range(len(words)):
      try:
        tensor[i] = all_words.index(words[i])
      except:
        print(i)
        raise
    return Variable(tensor)

print(word_tensor(['?', 'Caixeiro', 'extermínio']))

tensor([10253, 24476,  9563])


In [0]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set(): 
    chunk = random_chunk()
    inp = word_tensor(chunk[:-1])
    target = word_tensor(chunk[1:])
    return inp, target

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        size = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        return (size, size)

In [0]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str=['A'], predict_len=40, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = [h.cuda() for h in hidden]
    prime_input = word_tensor(prime_str).cuda()
    predicted = prime_str
    
    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_word = all_words[top_i]
        predicted.append(predicted_word)
        inp = word_tensor([predicted_word]).cuda()

    return ''.join(predicted)

In [0]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = [h.cuda() for h in hidden]
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [9]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_words, hidden_size, n_words, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate(['Os'], 40), '\n')


 12s (100 0%) 4.9789]
OsBotaseleçõesespereisobrenaturalmenteCorriAssadores os, Só, que de nome da minha com depois de quer a existiriam,   o. 

 25s (200 0%) 4.6550]
Osarticulaçãosilênciotransformar o que gente fornecedores garantem ! se? Continuava ―? que para mundo distribuir o a com era  

 38s (300 0%) 4.0497]
Osdeus
de que Não da ganhar seria o Mas mau-caráter reuniriam mesa. !, claro não de eram Foi Mas  

 51s (400 0%) 2.9817]
Osinsetos . Mercedes, alimenta que diz do o quando no dela do que na já que é que homem O 

 4s (500 1%) 3.1418]
Os. de os é o Para qualquer apática.  o o executiva e se como Uma. um . do os 

 17s (600 1%) 4.2620]
Os de e do primeira” e uniam. a com para ― ― que consternou a fazer se cada não ― 

 30s (700 1%) 3.8874]
Os. Que algum sua de verão. , mulher no camisa carteiras 
 diferença não mas , em aconselhar pode chegar 

 43s (800 1%) 3.8955]
Os papagaio menos de o entregador "são." ― vez Está pela o seguinte a origem mais. viúva não s

In [10]:
# Com a rede treinada, gerar um texto longo

print(evaluate(['O',' ', 'psicanalista', ' ', 'de'], 5500), '\n')

O psicanalista de conseguir andavam de ouro ― o primeiro soltos a minha música. Vou pensava que também pôde tá um dos precisam a banana de um mate? Você sabe que não fala?
― Se você não era um planeta.
― O que também aparecia em que ser perfeito. Collor e hoje a cadeia por mim tenha certeza que eram anos e várias possibilidades. São Seja de cultura e entrar à paisagem de dentro, pois qualquer assim sozinha que os mitos e os braços e o Binho dessa consciência de todas as Paris com a lei de ano que a volta de idade e ela entra a frio, que era um amigo impossível os dentes, dizia que não fosse uma guerra com a evolução do sangue É alguém também é o que é o seu deus?
— É uma crise e o chão diz que todos a América na cama, no ver do sol foi a Revolução humana. O sol tinham dificuldade com os netos e o buraco para comida. "O vizinho é o que eu sou o meu cérebro e eu não esqueci?" e o homem aceitarem um ao contrário não era o seu futebol desse que acho que não, dr.

##Discussão dos resultados

Os modelos utilizando GRU e LSTM a nível de caractere apresentaram resultados praticamente equivalentes, aparentemente com apenas uma ligeira vantagem da GRU em tempo de processamento. Ambas aprenderam rapidamente o uso de espaçamento e vogais, gradualmente evoluindo a formação de palavras e uso de pontuação e símbolos.

Ao mudar para nível de palavras, a mudança é muito mais acentuada, a começar pelas adaptações necessárias no código. Era possível antes alimentar a rede com um pequeno conjunto com todos os caracteres presentes no texto, assim ela mesma aprendia naturalmente a função de cada um em uma palavra ou frase. Para catalogar palavras, é necessário identificar o que é uma palavra e o que é um símbolo ou pontuação isoladamente. Ao fazer esta distinção, a rede consegue aprender como arquitetar frases formadas pelas palavras e símbolos que já conhece. Salvo erros por codificação incomum de caracteres, não serão geradas palavras que não existem, porém também é impossível a rede tentar criar novas combinações de letras que façam sentido, pois está limitada ao vocabulário que conhece.

Assim, a geração de texto a nível de palavras preocupou-se mais em aprender o uso de pontuação e estruturação de frases, além de ser possível perceber que muitas vezes há sequências de palavras que normalmente seriam vistas juntas na escrita humana
.